In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# High-level emission areas for the alpine hut
emission_sources = {
    'Helicopter Transport': {
        'num_trips': 10,  # Number of trips per season
        'fuel_per_trip_liters': 200,  # Liters of fuel per trip
        'emission_factor_kg_co2_per_liter': 2.5  # CO2 emissions per liter of fuel
    },
    'Food Supply Chain': {
        'total_weight_kg': 500,  # Total weight of food per season
        'avg_distance_km': 100,  # Average distance food travels
        'emission_factor_kg_co2_per_km_per_kg': 0.05  # CO2 emissions per km per kg of food
    },
    'Drink Supply Chain': {
        'total_weight_kg': 300,
        'avg_distance_km': 80,
        'emission_factor_kg_co2_per_km_per_kg': 0.04
    },
    'Heating': {
        'energy_type': 'propane',  # Type of energy (e.g., propane, biomass)
        'total_energy_consumed_liters': 1000,  # Liters of propane
        'emission_factor_kg_co2_per_liter': 1.6  # CO2 emissions per liter
    }
}

# Display the high-level structure
emission_sources

# Function to calculate helicopter transport emissions
def calculate_helicopter_emissions(params):
    return params['num_trips'] * params['fuel_per_trip_liters'] * params['emission_factor_kg_co2_per_liter']

# Function to calculate food supply chain emissions
def calculate_food_emissions(params):
    return params['total_weight_kg'] * params['avg_distance_km'] * params['emission_factor_kg_co2_per_km_per_kg']

# Function to calculate drink supply chain emissions
def calculate_drink_emissions(params):
    return params['total_weight_kg'] * params['avg_distance_km'] * params['emission_factor_kg_co2_per_km_per_kg']

# Function to calculate heating emissions
def calculate_heating_emissions(params):
    return params['total_energy_consumed_liters'] * params['emission_factor_kg_co2_per_liter']

def calculate_total_emissions(sources):
    total_emissions = 0
    total_emissions += calculate_helicopter_emissions(sources['Helicopter Transport'])
    total_emissions += calculate_food_emissions(sources['Food Supply Chain'])
    total_emissions += calculate_drink_emissions(sources['Drink Supply Chain'])
    total_emissions += calculate_heating_emissions(sources['Heating'])
    return total_emissions

# Calculate and display total emissions
total_emissions = calculate_total_emissions(emission_sources)
print(f"Total CO2 Emissions: {total_emissions:.2f} kg CO2")


Total CO2 Emissions: 10060.00 kg CO2


In [3]:
from ipywidgets import interact

# Interactive function to calculate total emissions based on user input
def interactive_emissions(num_trips=10, fuel_per_trip_liters=200, total_food_weight_kg=500, heating_fuel_liters=1000):
    emission_sources['Helicopter Transport']['num_trips'] = num_trips
    emission_sources['Helicopter Transport']['fuel_per_trip_liters'] = fuel_per_trip_liters
    emission_sources['Food Supply Chain']['total_weight_kg'] = total_food_weight_kg
    emission_sources['Heating']['total_energy_consumed_liters'] = heating_fuel_liters
    
    total_emissions = calculate_total_emissions(emission_sources)
    print(f"Total CO2 Emissions: {total_emissions:.2f} kg CO2")

# Create sliders for user interaction
interact(interactive_emissions, num_trips=(1, 20), fuel_per_trip_liters=(100, 300), total_food_weight_kg=(100, 1000), heating_fuel_liters=(500, 2000))


interactive(children=(IntSlider(value=10, description='num_trips', max=20, min=1), IntSlider(value=200, descri…

<function __main__.interactive_emissions(num_trips=10, fuel_per_trip_liters=200, total_food_weight_kg=500, heating_fuel_liters=1000)>